In [2]:
import yahoo_fin.stock_info as si
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from pandas import read_html

nan=np.nan

In [3]:
def GrahamInVal (sticker):
    #sticker = "CRM"

    # Read EPS
    url_Summary = 'https://finance.yahoo.com/quote/' + sticker
    dframe_list_Summary = pd.read_html(url_Summary)
    Summary_tab = dframe_list_Summary[1]
    EPS_str = Summary_tab.loc[3,1]
    EPS = float(EPS_str)

    # Read P/E
    #PERatio_str = Summary_tab.loc[2,1]
    #PERatio_read = float(PERatio_str)
    PERatio_fixed = 7.0
    PERatio = PERatio_fixed

    # Read 5 Yr Growth Rate
    url_Analysis = 'https://finance.yahoo.com/quote/' + sticker + '/analysis?p=' + sticker
    dframe_list_Analysis = pd.read_html(url_Analysis)
    GrowthEst_tab = dframe_list_Analysis[5]
    GrowthEst_str = GrowthEst_tab.loc[4,sticker]
    GrowthEst = float(GrowthEst_str[:-1])
    GrowthScale = 1

    # Read AAACorpBond Yield
    url_AAACorpBond = 'https://ycharts.com/indicators/moodys_seasoned_aaa_corporate_bond_yield'
    dframe_list_AAACorpBond = pd.read_html(url_AAACorpBond)
    dframe_AAACorpBond = dframe_list_AAACorpBond[0]
    AAACorpBond_str = dframe_AAACorpBond.loc[0,'Unnamed: 1']
    AAACorpBond = float(AAACorpBond_str[:-1])
    CorpBond = 4.4

    IntrinsicVal = round((EPS * (PERatio + (GrowthEst * GrowthScale)) * CorpBond) / AAACorpBond,2)

    # Read Current Val
    Summary_tab = dframe_list_Summary[0]
    PrevClose_str = Summary_tab.loc[0,1]
    PrevClose = round(float(PrevClose_str),2)

    if PrevClose < IntrinsicVal:
        Decision = 'BUY'
    else:
        Decision = 'SELL'

    MarginRaw = (PrevClose/IntrinsicVal)-1
    Margin = round(MarginRaw*100,2)
    #Margin = '{percent:.2%}'.format(percent=MarginRaw)

    return {'Sticker':sticker,
             'IntrinsicVal($)':IntrinsicVal,
             'PrevClose($)':PrevClose,
             'Decision':Decision,
             'Margin(%)':Margin,
             'EPS':EPS,
             'GrowthEst(%)':GrowthEst}

In [4]:
#DO NOT TOUCH THIS

#stock_val = GrahamInVal("AAPL")

# Create the pandas DataFrame 
#stock_tabsum = pd.DataFrame.from_dict(stock_val, orient='index')
#stock_sum = stock_tabsum.transpose()

# print dataframe. 
#stock_sum.set_index('Sticker', inplace=True)


In [14]:
stock_list = Series(["SNGX"])

#stock_sum = pd.DataFrame(np.array([[nan, nan, nan, nan, nan, nan, nan]]),
stock_sum = pd.DataFrame(np.arange(7*stock_list.size).reshape((stock_list.size,7)),
                         columns=['Sticker',
                                  'IntrinsicVal($)',
                                  'PrevClose($)',
                                  'Decision', 
                                  'Margin(%)',
                                  'EPS',
                                  'GrowthEst(%)'])

for i in stock_list:
    ami=GrahamInVal(i)
    ami1 = pd.DataFrame.from_dict(ami, orient='index')
    ami2 = ami1.transpose()
    #print(ami2)
    stock_sum = stock_sum.append(ami2)

stock_sum.set_index('Sticker', inplace=True)
stock_sum.drop(stock_sum.index[0:stock_list.size], inplace=True)
print(stock_sum)

        IntrinsicVal($) PrevClose($) Decision Margin(%)    EPS GrowthEst(%)
Sticker                                                                    
SNGX             -12.81         2.26     SELL   -117.64 -0.739            1
